# AdFlush Model
Test AdFlush ONNX, or train and test AdFlush upon H2O

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)
import time
import os
import seaborn as sns

In [ ]:
DATADIR=os.path.join(os.getcwd(),os.pardir,"dataset")
MODELDIR=os.path.join(os.getcwd(),os.pardir,"model")

Select the dataset to use during the session

In [ ]:
# Available datasets
testset=os.path.join(DATADIR, "AdFlush_test.csv")
GAN_testset=os.path.join(DATADIR, "GAN_mutated_AdFlush.csv")
gnirts_testset=os.path.join(DATADIR, "JS_obfuscated_gnirts.csv")
javascript_obfuscator_testset=os.path.join(DATADIR, "JS_obfuscated_javascript_obfuscator.csv")
wobfuscator_testset=os.path.join(DATADIR, "JS_obfuscated_wobfuscator.csv")

# Select dataset to import
DATASET=testset
ISMUTATED=False
if DATASET==testset or DATASET==GAN_testset:
    data_df=pd.read_csv(DATASET, index_col=0)
elif DATASET==gnirts_testset or DATASET==javascript_obfuscator_testset or DATASET==wobfuscator_testset:
    js_features=['brackettodot', 'num_get_storage', 'num_set_storage',
        'num_get_cookie', 'num_requests_sent', 'ng_0_0_2', 'ng_0_15_15', 'ng_2_13_2',
        'ng_15_0_3', 'ng_15_0_15', 'ng_15_15_15', 'avg_ident',
        'avg_charperline']
    test_df=pd.read_csv(testset,index_col=0)
    mutate_df=pd.read_csv(DATASET,index_col=0)
    mutate_df=mutate_df[js_features].copy()
    mutate_performed=mutate_df.index
    test_df=test_df.loc[mutate_performed].copy()
    test_df.loc[mutate_performed,js_features]=mutate_df.loc[mutate_performed,js_features]
    data_df=test_df.copy()
    ISMUTATED=True
label=data_df['label'].astype(int)

print("Using Dataset: ",DATASET)
data_df

In [ ]:
def metrics(true, pred, _is_mutated, _return):
    if _return:
        # Number of attacks
        total_attacks = len(true)
        
        # Number of successful attacks (misclassifications)
        successful_attacks = sum(true != pred)
        tn, fp, fn, tp = confusion_matrix(true, pred).ravel()

        # Calculate FNR
        fnr = fn / (tp + fn)

        # Calculate FPR
        fpr = fp / (fp + tn)
        return accuracy_score(true, pred), precision_score(true, pred), recall_score(true, pred), f1_score(true, pred), fnr, fpr
    
    else:        
        print(f"Accuracy : {accuracy_score(true, pred)} ")
        print(f"Precision : {precision_score(true, pred)} ")
        print(f"Recall : {recall_score(true, pred)} ")
        print(f"F1 : {f1_score(true, pred)} ")
        # Number of attacks
        total_attacks = len(true)
        
        # Number of successful attacks (misclassifications)
        successful_attacks = sum(true != pred)
        tn, fp, fn, tp = confusion_matrix(true, pred).ravel()

        # Calculate FNR
        fnr = fn / (tp + fn)
        print('False Negative Rate:', fnr)

        # Calculate FPR
        fpr = fp / (fp + tn)
        print('False Positive Rate:', fpr)
        # ASR
        if _is_mutated:
            asr = successful_attacks / total_attacks
            print("Attack Success Rate: ",asr)

### 1. AdFlush_ONNX

In [ ]:
import onnx
import onnxruntime as ort
import onnxmltools

In [ ]:
print('Loading Model')
model = onnx.load(os.path.join(MODELDIR,'AdFlush.onnx'))
# Check that the IR is well formed
try:
    print("Checking model integrity")
    onnx.checker.check_model(model)
    # Create an ONNX runtime session
    ort_session = ort.InferenceSession(os.path.join(MODELDIR,'AdFlush.onnx'))
    input_data = data_df.drop('label', axis=1).values.astype('float32')
    input_name = ort_session.get_inputs()[0].name
    label_name = ort_session.get_outputs()[0].name

    # Run the inference session to get the prediction results
    print('Running Inference Session')
    start_time=time.time()
    pred = ort_session.run([label_name], {input_name: input_data})[0]
    print("Inference time elapsed: ", time.time()-start_time, "seconds for ", len(label), " samples.")
    metrics(label.astype(int),pred.astype(int),ISMUTATED,False)
    print("AUROC: ",roc_auc_score(label, pred.astype('float64')))
    
except Exception as e:
    print("Error in loading model: ",e)

### 2. AdFlush_H2O_MOJO

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads = 12, max_mem_size = "64g", enable_assertions = False)
h2o.no_progress()
custom_path=""

#### 2-1. Making a new AdFlush with H2O AutoML (optional)

##### 2-1-1. Training custom AdFlush

In [ ]:
train=pd.read_csv(os.path.join(DATADIR,'AdFlush_train.csv'),index_col=0)
test=data_df.copy() #Use selected Dataset to test
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

y = 'label'
x = list(train)
x.remove(y)

h2o_train = h2o.H2OFrame(train)
h2o_test = h2o.H2OFrame(test)

h2o_train[y] = h2o_train[y].asfactor()
h2o_test[y] = h2o_test[y].asfactor()

# Select runtime for exploration
MAXRUNTIME=3600
aml = H2OAutoML(max_runtime_secs=MAXRUNTIME, max_models=None, exclude_algos=['XGBoost', 'StackedEnsemble'], nfolds=5)
# Run below code instead if you want to convert to ONNX format.
# aml = H2OAutoML(max_runtime_secs=100, max_models=None,include_algos=['GBM'], nfolds=5)

aml.train(x = x, y = y, training_frame = h2o_train, leaderboard_frame = h2o_test)
custom_path=aml.leader.save_mojo(os.path.join(MODELDIR,'AdFlush_custom'))
custom_path

##### 2-1-2. Convert custom AdFlush to ONNX

In [ ]:
## Only available with GBM model
## We found that there is a problem in converting h2o to onnx format in windows due to automatically caching to ~AppData\temp, thus recommend to run this code in Linux
onnx_model = onnxmltools.convert.convert_h2o(custom_path, target_opset=9)
onnxmltools.utils.save_model(onnx_model, os.path.join(MODELDIR, "AdFlush_custom.onnx"))

#### 2-2. Testing AdFlush with H20 AutoML

In [ ]:
path =os.path.join(MODELDIR,"AdFlush_mojo.zip")

## Run code below to use custom AdFlush
# path=custom_path

print("Loading model...")
h2o_model = h2o.import_mojo(path)
h2o_test=h2o.H2OFrame(data_df)

start_time=time.time()
pred = h2o_model.predict(h2o_test)
pred = pred.as_data_frame().predict.to_list()
print("Inference time elapsed: ", time.time()-start_time, "seconds for ", len(label), " samples.")
metrics(label, pred, ISMUTATED, False)
if ISMUTATED:
    print("AUROC: ",roc_auc_score(label, pred.astype('float64')))

#### 2-3.Explanable values for AdFlush H2O AutoML
We can see some explanations of the processed AdFlush model using H2O AutoML,
including SHAP values.

In [ ]:
h2o_model

In [ ]:
h2o_model.shap_summary_plot(h2o_test)

In [ ]:
h2o.cluster().shutdown()

### 2-4. Longitudinal Inference

In [ ]:
longitude_labels=pd.read_csv(os.path.join(DATADIR,'longitude_label.csv'),index_col=0)

In [ ]:
long_acc=[]
long_pre=[]
long_rec=[]
long_f1=[]
long_fnr=[]
long_fpr=[]

DATES=[
    '0406', '0410', '0414', '0418', '0422', '0426', '0430',
    '0504', '0508', '0512', '0516', '0520', '0524', '0528', '0601', '0605',
    '0609', '0613', '0617', '0621', '0625', '0629', '0703', '0707', '0711',
    '0715', '0719', '0723', '0727', '0731', '0804', '0808', '0812', '0816',
    '0820', '0824', '0828', '0901', '0905', '0909', '0913', '0917'
]

if DATASET!=testset:
    DATASET=testset
    data_df=pd.read_csv(DATASET, index_col=0)
    path =os.path.join(MODELDIR,"AdFlush_mojo.zip")
    print("Loading model...")
    h2o_model = h2o.import_mojo(path)
    h2o_test=h2o.H2OFrame(data_df)

    start_time=time.time()
    pred = h2o_model.predict(h2o_test)
    pred = pred.as_data_frame().predict.to_list()
    
for date in DATES:
    acc, pre, rec, f1, fnr, fpr=metrics(longitude_labels[date], pred, ISMUTATED, True)
    long_acc.append(acc)
    long_pre.append(pre)
    long_rec.append(rec)
    long_f1.append(f1)
    long_fnr.append(fnr)
    long_fpr.append(fpr)

longitude_result=pd.DataFrame()
longitude_result['accuracy']=long_acc
longitude_result['precision']=long_pre
longitude_result['recall']=long_rec
longitude_result['f1_score']=long_f1
longitude_result['false_negative_rate']=long_fnr
longitude_result['false_positive_rate']=long_fpr
longitude_result.index=DATES
longitude_result

In [ ]:
longitude_result.max(), longitude_result.min()

In [ ]:
lineplot=sns.lineplot(longitude_result[['accuracy','precision','recall','f1_score']])
plt.xticks(rotation=45)
plt.figure(figsize=(40,30), dpi=100)
lineplot.set_ylim(0.92,0.985)
lineplot.figure.set_size_inches(12,6)